# Database Check
Validate raw + derived tables and provenance for the FINRA + Polygon pipeline.


In [19]:
from darkpool_analysis.config import load_config
from darkpool_analysis.db import get_connection
import pandas as pd

config = load_config()
conn = get_connection(config.db_path)


In [20]:
tables = conn.execute("SHOW TABLES").df()["name"].tolist()
tables


['composite_signal',
 'daily_metrics',
 'finra_otc_weekly_raw',
 'finra_short_daily_raw',
 'index_constituent_short_agg_daily',
 'lit_direction_daily',
 'polygon_daily_agg_raw',
 'polygon_equity_trades_raw']

In [21]:
from IPython.display import display

for table in tables:
    print(table)
    display(conn.execute(f"DESCRIBE {table}").df())
    display(conn.execute(f"SELECT COUNT(*) AS n FROM {table}").df())


composite_signal


,column_name,column_type,null,key,default,extra
0,symbol,VARCHAR,YES,None,None,None
1,date,DATE,YES,None,None,None
2,directional_score,DOUBLE,YES,None,None,None
3,pressure_score,DOUBLE,YES,None,None,None
4,participation_score,DOUBLE,YES,None,None,None
5,composite_score,DOUBLE,YES,None,None,None
6,final_label,VARCHAR,YES,None,None,None
7,data_quality,VARCHAR,YES,None,None,None
8,inference_version,VARCHAR,YES,None,None,None


,n
0,0


daily_metrics


,column_name,column_type,null,key,default,extra
0,date,DATE,YES,None,None,None
1,symbol,VARCHAR,YES,None,None,None
2,log_buy_sell,DOUBLE,YES,None,None,None
3,short_volume,DOUBLE,YES,None,None,None
4,short_exempt_volume,DOUBLE,YES,None,None,None
5,short_total_volume,DOUBLE,YES,None,None,None
6,short_ratio,DOUBLE,YES,None,None,None
7,short_ratio_z,DOUBLE,YES,None,None,None
8,short_ratio_denominator_type,VARCHAR,YES,None,None,None
9,short_ratio_denominator_value,DOUBLE,YES,None,None,None


...

,column_name,column_type,null,key,default,extra
0,symbol,VARCHAR,YES,None,None,None
1,trade_date,DATE,YES,None,None,None
2,open,DOUBLE,YES,None,None,None
3,high,DOUBLE,YES,None,None,None
4,low,DOUBLE,YES,None,None,None
5,close,DOUBLE,YES,None,None,None
6,vwap,DOUBLE,YES,None,None,None
7,volume,DOUBLE,YES,None,None,None


,n
0,10


polygon_equity_trades_raw


,column_name,column_type,null,key,default,extra
0,symbol,VARCHAR,YES,None,None,None
1,timestamp,TIMESTAMP,YES,None,None,None
2,price,DOUBLE,YES,None,None,None
3,size,DOUBLE,YES,None,None,None
4,bid,DOUBLE,YES,None,None,None
5,ask,DOUBLE,YES,None,None,None


,n
0,4892650


In [22]:
sample_symbol = "AMZN"
start_date = min(config.target_dates)
end_date = max(config.target_dates)
sample_symbol, start_date, end_date


('AMZN', datetime.date(2025, 11, 14), datetime.date(2025, 12, 22))

## Raw Tables Samples


In [23]:
conn.execute("""
SELECT * FROM finra_otc_weekly_raw
WHERE symbol = ?
ORDER BY week_start_date DESC
LIMIT 5
""", [sample_symbol]).df()


,symbol,week_start_date,off_exchange_volume,trade_count,tier_identifier,tier_description,issue_name,market_participant_name,mpid,last_update_date,source_file
0,AMZN,2025-11-10,1102480.0,2739.0,T1,NMS Tier 1,"Amazon.com, Inc. Common Stock",BIDS BIDS ATS,BIDS,2025-12-01,None
1,AMZN,2025-11-10,2215.0,112.0,T1,NMS Tier 1,"Amazon.com, Inc. Common Stock","BOSS BRUCE MARKETS, LLC.",BOSS,2025-12-01,None


In [24]:
conn.execute("""
SELECT * FROM finra_short_daily_raw
WHERE symbol = ? AND trade_date BETWEEN ? AND ?
ORDER BY trade_date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,trade_date,short_volume,short_exempt_volume,total_volume,market,source,source_file
0,AMZN,2025-12-22,52397.0,0.0,133850.0,B,None,None
1,AMZN,2025-12-22,3656129.0,44098.0,11173610.0,Q,None,None
2,AMZN,2025-12-22,209803.0,0.0,505076.0,N,None,None
3,AMZN,2025-12-19,53483.0,0.0,131418.0,B,None,None
4,AMZN,2025-12-19,5011154.0,113743.0,15696839.0,Q,None,None


In [25]:
conn.execute("""
SELECT * FROM polygon_daily_agg_raw
WHERE symbol = ? AND trade_date BETWEEN ? AND ?
ORDER BY trade_date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,trade_date,open,high,low,close,vwap,volume
0,AMZN,2025-12-22,228.610,229.480,226.7100,228.43,228.1121,32189837.0
1,AMZN,2025-12-19,226.760,229.125,225.5800,227.35,227.4216,85539391.0
2,AMZN,2025-12-18,225.705,229.225,224.4100,226.76,226.8046,50267599.0
3,AMZN,2025-12-17,224.655,225.190,220.9900,221.27,222.5990,44030569.0
4,AMZN,2025-12-16,223.035,223.660,221.1304,222.56,222.6031,39293678.0


In [26]:
conn.execute("""
SELECT * FROM polygon_equity_trades_raw
WHERE symbol = ? AND timestamp::DATE BETWEEN ? AND ?
ORDER BY timestamp DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,timestamp,price,size,bid,ask
0,AMZN,2025-12-22 23:59:57.336425,228.600,141.0,NaN,NaN
1,AMZN,2025-12-22 23:59:57.336424,228.600,31.0,NaN,NaN
2,AMZN,2025-12-22 23:59:52.105948,228.595,1.0,NaN,NaN
3,AMZN,2025-12-22 23:59:51.591104,228.595,1.0,NaN,NaN
4,AMZN,2025-12-22 23:59:37.652689,228.590,2.0,NaN,NaN


## Derived Tables Samples


In [27]:
conn.execute("""
SELECT * FROM lit_direction_daily
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,date,lit_buy_volume,lit_sell_volume,lit_buy_ratio,log_buy_sell,classification_method,lit_coverage_pct,inference_version
0,AMZN,2025-12-22,6528473.0,6689623.0,0.493904,-0.024384,TICK,48.7161,PhaseA_v1
1,AMZN,2025-12-19,9556591.0,47899508.0,0.166329,-1.611874,TICK,49.6359,PhaseA_v1
2,AMZN,2025-12-18,8990006.0,23658198.0,0.275360,-0.967596,TICK,50.7755,PhaseA_v1
3,AMZN,2025-12-17,6776948.0,21542940.0,0.239300,-1.156521,TICK,52.0088,PhaseA_v1
4,AMZN,2025-12-16,17328990.0,6901810.0,0.715164,0.920597,TICK,50.2187,PhaseA_v1


In [28]:
conn.execute("""
SELECT * FROM daily_metrics
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
LIMIT 10
""", [sample_symbol, start_date, end_date]).df()


,date,symbol,log_buy_sell,short_volume,short_exempt_volume,short_total_volume,short_ratio,short_ratio_z,short_ratio_denominator_type,short_ratio_denominator_value,...,range_pct,otc_off_exchange_volume,otc_week_used,data_quality,has_otc,has_short,has_lit,has_price,pressure_context_label,inference_version
0,2025-12-22,AMZN,-0.024384,3918329.0,44098.0,11812536.0,0.335443,-0.052630,FINRA_TOTAL,11812536.0,...,0.012126,1104695.0,2025-11-10,PRE_OTC,True,True,True,True,NEUTRAL,PhaseA_v1
1,2025-12-19,AMZN,-1.611874,5638086.0,116189.0,17894670.0,0.321564,-0.423604,FINRA_TOTAL,17894670.0,...,0.015593,1104695.0,2025-11-10,PRE_OTC,True,True,True,True,NEUTRAL,PhaseA_v1
2,2025-12-18,AMZN,-0.967596,3966284.0,21747.0,15645480.0,0.254900,-2.140477,FINRA_TOTAL,15645480.0,...,0.021234,1104695.0,2025-11-10,PRE_OTC,True,True,True,True,LOW_SHORT_STRONG_UP,PhaseA_v1
3,2025-12-17,AMZN,-1.156521,4844182.0,32677.0,13704311.0,0.355863,0.170750,FINRA_TOTAL,13704311.0,...,0.018981,1104695.0,2025-11-10,PRE_OTC,True,True,True,True,NEUTRAL,PhaseA_v1
4,2025-12-16,AMZN,0.920597,4359030.0,22641.0,12862955.0,0.340643,-0.383693,FINRA_TOTAL,12862955.0,...,0.011366,1104695.0,2025-11-10,PRE_OTC,True,True,True,True,NEUTRAL,PhaseA_v1
5,2025-12-15,AMZN,-0.715683,4691250.0,199677.0,15459200.0,0.316376,-1.266603,FINRA_TOTAL,15459200.0,...,0.028894,1104695.0,2025-11-10,PRE_OTC,True,True,True,True,LOW_SHORT_SELL_OFF,PhaseA_v1
6,2025-12-12,AMZN,0.618450,5222697.0,29206.0,14190483.0,0.370100,NaN,FINRA_TOTAL,14190483.0,...,0.021928,1104695.0,2025-11-10,PRE_OTC,True,True,True,True,NEUTRAL,PhaseA_v1
7,2025-12-11,AMZN,0.241839,4442395.0,34141.0,11301782.0,0.396091,NaN,FINRA_TOTAL,11301782.0,...,0.014851,1104695.0,2025-11-10,PRE_OTC,True,True,True,True,NEUTRAL,PhaseA_v1
8,2025-12-10,AMZN,0.395790,5526771.0,23044.0,15620219.0,0.355297,NaN,FINRA_TOTAL,15620219.0,...,0.017085,1104695.0,2025-11-10,PRE_OTC,True,True,True,True,NEUTRAL,PhaseA_v1
9,2025-12-09,AMZN,-0.600199,3072148.0,18802.0,9453135.0,0.326976,NaN,FINRA_TOTAL,9453135.0,...,0.015181,1104695.0,2025-11-10,PRE_OTC,True,True,True,True,NEUTRAL,PhaseA_v1


In [29]:
conn.execute("""
SELECT index_symbol, trade_date, coverage_count, expected_constituent_count, coverage_pct
FROM index_constituent_short_agg_daily
ORDER BY trade_date DESC
LIMIT 5
""").df()


,index_symbol,trade_date,coverage_count,expected_constituent_count,coverage_pct
0,SPX,2025-12-22,1,10,0.1
1,SPX,2025-12-19,1,10,0.1
2,SPX,2025-12-18,1,10,0.1
3,SPX,2025-12-17,1,10,0.1
4,SPX,2025-12-16,1,10,0.1


## Coverage and Provenance Checks


In [30]:
conn.execute("""
SELECT symbol, COUNT(*) AS rows
FROM daily_metrics
WHERE date BETWEEN ? AND ?
GROUP BY symbol
ORDER BY symbol
""", [start_date, end_date]).df()


,symbol,rows
0,AMZN,10


In [31]:
conn.execute("""
SELECT date, symbol, otc_off_exchange_volume, otc_week_used, data_quality
FROM daily_metrics
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
""", [sample_symbol, start_date, end_date]).df()


,date,symbol,otc_off_exchange_volume,otc_week_used,data_quality
0,2025-12-22,AMZN,1104695.0,2025-11-10,PRE_OTC
1,2025-12-19,AMZN,1104695.0,2025-11-10,PRE_OTC
2,2025-12-18,AMZN,1104695.0,2025-11-10,PRE_OTC
3,2025-12-17,AMZN,1104695.0,2025-11-10,PRE_OTC
4,2025-12-16,AMZN,1104695.0,2025-11-10,PRE_OTC
5,2025-12-15,AMZN,1104695.0,2025-11-10,PRE_OTC
6,2025-12-12,AMZN,1104695.0,2025-11-10,PRE_OTC
7,2025-12-11,AMZN,1104695.0,2025-11-10,PRE_OTC
8,2025-12-10,AMZN,1104695.0,2025-11-10,PRE_OTC
9,2025-12-09,AMZN,1104695.0,2025-11-10,PRE_OTC


## Sanity Checks


In [32]:
conn.execute("""
SELECT symbol, date, lit_buy_volume, lit_sell_volume, log_buy_sell
FROM lit_direction_daily
WHERE (lit_buy_volume <= 0 OR lit_sell_volume <= 0)
  AND log_buy_sell IS NOT NULL
LIMIT 10
""").df()


,symbol,date,lit_buy_volume,lit_sell_volume,log_buy_sell


In [33]:
conn.execute("""
SELECT symbol, date, short_ratio, short_ratio_denominator_type
FROM daily_metrics
WHERE short_ratio_denominator_type IS NOT NULL
  AND (short_ratio < 0 OR short_ratio > 1)
LIMIT 10
""").df()


,symbol,date,short_ratio,short_ratio_denominator_type


In [34]:
conn.execute("""
SELECT symbol, date, COUNT(*) AS n
FROM daily_metrics
GROUP BY symbol, date
HAVING n > 1
""").df()


,symbol,date,n


In [35]:
conn.execute("""
SELECT symbol, date, COUNT(*) AS n
FROM lit_direction_daily
GROUP BY symbol, date
HAVING n > 1
""").df()


,symbol,date,n


In [36]:
conn.close()
